# Welcome

## Package preparation

Follow README.md first.



In [1]:
import sys
import os
#!conda install --yes --prefix {sys.prefix} numpy
#!conda install --yes --prefix {sys.prefix} seaborn
#!{sys.executable} -m pip install seaborn
#!{sys.executable} -m pip install plotly
#!{sys.executable} -m pip install cufflinks
#!{sys.executable} -m pip install ipywidgets

import seaborn as pkg
pkg.__path__

['/Users/tmhoangtus.ibm.com/anaconda3/envs/py37/lib/python3.7/site-packages/seaborn']

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import matplotlib as plt
import numpy as np
import pandas as pd
import json
import pprint
from request_openfda import *
from pandas.io.json import json_normalize
import yaml
# print(yaml.__version__)
import urllib.request

pp = pprint.PrettyPrinter(indent=4)
sns.set_context(context='talk')

pkmn_type_colors = ['#78C850',  # Grass
                    '#F08030',  # Fire
                    '#6890F0',  # Water
                    '#A8B820',  # Bug
                    '#A8A878',  # Normal
                    '#A040A0',  # Poison
                    '#F8D030',  # Electric
                    '#E0C068',  # Ground
                    '#EE99AC',  # Fairy
                    '#C03028',  # Fighting
                    '#F85888',  # Psychic
                    '#B8A038',  # Rock
                    '#705898',  # Ghost
                    '#98D8D8',  # Ice
                    '#7038F8',  # Dragon
                   ]

## Essential files preparation 

1. Get the searchable fields 

```console
https://open.fda.gov/apis/drug/event/searchable-fields
```
  

In [3]:

url="https://open.fda.gov/fields/drugevent.yaml"
    
file_name = r"./drugs_fields.yaml"
try:
    if os.path.getsize(file_name) > 0:
        # Non empty file exists
        pass
    else:
        # Empty file exists
        urllib.request.urlretrieve(url, file_name)
except OSError as e:
    urllib.request.urlretrieve(url, file_name)

with open(file_name) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    attr_list = yaml.load(file, Loader=yaml.FullLoader)

searchable_fields = []
print_level = None
# print_level = 3 
get_searchable_fields(searchable_fields, attr_list, print_level=print_level)
print("Num fields: ", len(searchable_fields))
pp.pprint(searchable_fields)

Num fields:  86
[   'authoritynumb',
    'companynumb',
    'duplicate',
    'fulfillexpeditecriteria',
    'occurcountry',
    'patient.drug.actiondrug',
    'patient.drug.activesubstance.activesubstancename',
    'patient.drug.drugadditional',
    'patient.drug.drugadministrationroute',
    'patient.drug.drugauthorizationnumb',
    'patient.drug.drugbatchnumb',
    'patient.drug.drugcharacterization',
    'patient.drug.drugcumulativedosagenumb',
    'patient.drug.drugcumulativedosageunit',
    'patient.drug.drugdosageform',
    'patient.drug.drugdosagetext',
    'patient.drug.drugenddate',
    'patient.drug.drugenddateformat',
    'patient.drug.drugindication',
    'patient.drug.drugintervaldosagedefinition',
    'patient.drug.drugintervaldosageunitnumb',
    'patient.drug.drugrecurreadministration',
    'patient.drug.drugrecurrence',
    'patient.drug.drugseparatedosagenumb',
    'patient.drug.drugstartdate',
    'patient.drug.drugstartdateformat',
    'patient.drug.drugstructuredos

2. Get the country codes [NOTE: certain countries whose sovereignty is another country, such as Åland Islands (AX) is part of Finland (FI)]

In [4]:
url="https://datahub.io/core/country-list/r/data.csv"
file_name = r"country_codes.csv"
try:
    if os.path.getsize(file_name) > 0:
        # Non empty file exists
        pass
    else:
        # Empty file exists
        urllib.request.urlretrieve(url, file_name)
except OSError as e:
    urllib.request.urlretrieve(url, file_name)
   
df_country_code = pd.read_csv(file_name)
print("ISO 3166-1 standard currently comprises %i countries " % df_country_code.shape[0])
df_country_code.head()

ISO 3166-1 standard currently comprises 249 countries 


,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


OpenFDA provides an endpoint to get all data files. Data for "drug" has 4 groups

1. enforcement
2. ndc
3. event
4. label

We're interested in 'event'. 

REMARK: 
1. Checking the data for 'event', I'm not sure what "all other data" means. May be it refers to those from a different time window not from 2004Q1 to 2019Q3?

```console 
// https://api.fda.gov/download.json
"event": {
        "total_records": 9059581,
        "export_date": "2019-12-20",
        "partitions": [
          {
            "size_mb": "50.56",
            "records": 100000,
            "display_name": "All other data (part 1 of 3)",
            "file": "https://download.open.fda.gov/device/event/all_other/device-event-0001-of-0003.json.zip"
          },
```

2. Data are too big to download to local machine for now. However, we can download some files for checking its content. RestAPI is also available to use.

Here, I try to understand the kind of data in OpenFDA's drug-events, before transforming the data to answer the following questions [at the end of this notebook].

The kinds of questions of interest:

 Are different adverse events reported in different countries? 

 What are the different adverse events associated with different disease areas? 

 What drugs tend to be taken together?

In [5]:
# Try to download data files
years = None
years = ["2019 Q1"]
# years = [2005, "2019"]
years = ["2015 Q1"]
# --> list of years to download, otherwise download all
# download_data_files(years)



Check the file content of one json.zip file, AND find a good strategy to map JSON-based records into pd.DataFrame for data analysis.

In [6]:
# inspect a single zipfile
# and find a way to collapse from JSON nested structure into pd.DataFrame
# -> use orderedDict is better (if py < 3.6)
import zipfile  
from collections import OrderedDict

# test_file = "./data/2004 Q1 (part 1 of 5)/drug-event-0001-of-0005.json.zip"
test_file = "./data/2019 Q1 (part 1 of 32)/drug-event-0001-of-0032.json.zip"
with zipfile.ZipFile(test_file, "r") as z:
   for filename in z.namelist():  
      print("File: ", test_file)  
      with z.open(filename) as f:  
         data = f.read()  
         # json_data = json.loads(data.decode("utf-8"), object_pairs_hook=OrderedDict) 
         json_data = json.loads(data.decode("utf-8"))
        
# pp.pprint(json_data)

all_records = json_data["results"]

count_primarysource = 0
count_primarysourcecountry = 0
count_occurcountry = 0
for rec in all_records:
    if "primarysource" in rec.keys():
        count_primarysource += 1
    if "primarysourcecountry" in rec.keys():
        count_primarysourcecountry += 1
    if "occurcountry" in rec.keys():
        count_occurcountry += 1
print("total recs having 'primarysource' ", count_primarysource)
print("total recs having 'primarysourcecountry' ", count_primarysourcecountry)
print("total recs having 'occurcountry' ", count_occurcountry)

records = json_data["results"][0:5]

record = json_data["results"][4]

print("patient.reaction ", record["patient"]["reaction"])

print("=======")
print("Root keys: ", record.keys())
# We should drop certain not-important keys
# ... some request for json_normalize but not merged yet: https://github.com/pandas-dev/pandas/pull/27262/files
# --> a better replacement is 'flatten_json' package, though it does not provide all features that I expect
root_keys = \
 ["receiptdate",
  "transmissiondate",
  "transmissiondateformat",
  "receiptdateformat",
  "receivedateformat",
  "fulfillexpeditecriteria",
  "safetyreportid",
  "companynumb",
  "sender"  # "sender.senderorganization"
 ]

def delete_root_keys(record, root_keys):
    for key in root_keys:
        try:
            del record[key]
        except KeyError:
            pass
        
print("=======")
delete_root_keys(record, root_keys)
print("(trimmed) Root keys: ", record.keys())
print("=======")

# Some second-level keys can be removed 
# .  sender
# .  primarysource
# .  patient
print("Second-level keys")
for x in record.keys():
    if isinstance(record[x], dict):
        print(" ", x)
print("primarysource ", record["primarysource"])
print("patient ", record["patient"])
# There are many subfield in 'patient' that we can ignore
ignore_fields = ["patientonsetageunit", "drugadministrationroute", 
                 "drugstartdateformat", "drugenddateformat",
                "drugdosagetext", "unii",
                "rxcui", "spl_set_id",
                "product_ndc", "spl_id",
                "application_number", "package_ndc",
                "drugbatchnumb",
                "drugstartdate", "drugenddate",
                "nui", "drugauthorizationnumb"
                ]

def delete_sub_keys(record, ignore_fields):
    all_keys = list(record.keys())
    for key in all_keys:
        if key in ignore_fields:
            del record[key]
    for key, val in record.items():
        if isinstance(val, dict):
            delete_sub_keys(val, ignore_fields)
        if isinstance(val, list):
            for x in val:
                if isinstance(x, dict):
                    delete_sub_keys(x, ignore_fields)
    pass

delete_sub_keys(record, ignore_fields)
print("=======")
print("... after deleting some fields")
print(record["patient"])
print("=======")
print("=======")
#print(record["patient"]["reaction"])

from flatten_json import flatten
flat_dic = flatten(record, '.')
flat_dics = []

# check if the flatterned records have the same number of columns?
# . As not all the records have the same subfields being used, it's quite challenge to
# .   ensure the processed records have the same number of fields so that it can be flattened
for record in records:
    flat_dic = flatten(record, '.')
    flat_dics.append(flat_dic)
    print("num columns: ", len(flat_dic.keys()))
    
# REMARKS: 'flatten_json' package does not keep the last list as a single entry, instead, it breaks into several elements
# . ... 
dic_flattened = (flatten(d) for d in records)
import pandas as pd
df_records = pd.DataFrame(dic_flattened)

pd.options.display.max_columns = None
display(df_records)
#df.head()


File:  ./data/2019 Q1 (part 1 of 32)/drug-event-0001-of-0032.json.zip
total recs having 'primarysource'  12000
total recs having 'primarysourcecountry'  11734
total recs having 'occurcountry'  11999
patient.reaction  [{'reactionmeddrapt': 'Premature separation of placenta', 'reactionmeddraversionpt': '21.1', 'reactionoutcome': '6'}, {'reactionmeddrapt': 'Off label use', 'reactionmeddraversionpt': '21.1', 'reactionoutcome': '6'}, {'reactionmeddrapt': 'Product use issue', 'reactionmeddraversionpt': '21.1', 'reactionoutcome': '6'}]
Root keys:  dict_keys(['receivedate', 'seriousnessother', 'patient', 'sender', 'primarysourcecountry', 'transmissiondate', 'companynumb', 'occurcountry', 'receiptdate', 'transmissiondateformat', 'receiptdateformat', 'primarysource', 'duplicate', 'reporttype', 'reportduplicate', 'receiver', 'serious', 'safetyreportversion', 'fulfillexpeditecriteria', 'safetyreportid', 'receivedateformat'])
(trimmed) Root keys:  dict_keys(['receivedate', 'seriousnessother', 'pati

,receivedate,receivedateformat,patient_reaction_0_reactionmeddrapt,patient_reaction_0_reactionmeddraversionpt,patient_reaction_0_reactionoutcome,patient_patientsex,patient_drug_0_medicinalproduct,patient_drug_0_drugindication,patient_drug_0_drugcharacterization,patient_drug_0_drugadministrationroute,patient_drug_0_drugseparatedosagenumb,patient_drug_0_drugstructuredosageunit,patient_drug_0_openfda_manufacturer_name_0,patient_drug_0_openfda_unii_0,patient_drug_0_openfda_product_type_0,patient_drug_0_openfda_rxcui_0,patient_drug_0_openfda_rxcui_1,patient_drug_0_openfda_spl_set_id_0,patient_drug_0_openfda_route_0,patient_drug_0_openfda_generic_name_0,patient_drug_0_openfda_brand_name_0,patient_drug_0_openfda_product_ndc_0,patient_drug_0_openfda_substance_name_0,patient_drug_0_openfda_spl_id_0,patient_drug_0_openfda_application_number_0,patient_drug_0_openfda_package_ndc_0,patient_drug_0_openfda_package_ndc_1,patient_drug_0_openfda_package_ndc_2,patient_drug_0_drugbatchnumb,patient_drug_0_drugintervaldosageunitnumb,patient_drug_0_drugdosagetext,patient_drug_0_actiondrug,patient_drug_0_activesubstance_activesubstancename,patient_drug_0_drugintervaldosagedefinition,patient_drug_0_drugauthorizationnumb,patient_drug_0_drugdosageform,patient_drug_0_drugstructuredosagenumb,sender_senderorganization,sender_sendertype,transmissiondate,fulfillexpeditecriteria,occurcountry,receiptdate,transmissiondateformat,receiptdateformat,primarysource_reportercountry,primarysource_qualification,duplicate,reporttype,reportduplicate_duplicatenumb,reportduplicate_duplicatesource,receiver_receiverorganization,receiver_receivertype,serious,safetyreportversion,companynumb,safetyreportid,primarysourcecountry,seriousnessother,patient_reaction_1_reactionmeddrapt,patient_reaction_1_reactionmeddraversionpt,patient_reaction_1_reactionoutcome,patient_reaction_2_reactionmeddrapt,patient_reaction_2_reactionmeddraversionpt,patient_reaction_2_reactionoutcome,patient_reaction_3_reactionmeddrapt,patient_reaction_3_reactionmeddraversionpt,patient_reaction_3_reactionoutcome,patient_reaction_4_reactionmeddrapt,patient_reaction_4_reactionmeddraversionpt,patient_reaction_4_reactionoutcome,patient_patientonsetage,patient_drug_0_openfda_rxcui_2,patient_drug_0_openfda_rxcui_3,patient_drug_0_openfda_rxcui_4,patient_drug_0_openfda_rxcui_5,patient_drug_0_openfda_rxcui_6,patient_drug_0_openfda_rxcui_7,patient_drug_0_openfda_rxcui_8,patient_drug_0_openfda_rxcui_9,patient_drug_0_openfda_rxcui_10,patient_drug_0_openfda_rxcui_11,patient_drug_0_openfda_rxcui_12,patient_drug_0_openfda_rxcui_13,patient_drug_0_openfda_rxcui_14,patient_drug_0_openfda_rxcui_15,patient_drug_0_openfda_rxcui_16,patient_drug_0_openfda_rxcui_17,patient_drug_0_openfda_rxcui_18,patient_drug_0_openfda_rxcui_19,patient_drug_0_openfda_rxcui_20,patient_drug_0_openfda_rxcui_21,patient_drug_0_openfda_rxcui_22,patient_drug_0_openfda_rxcui_23,patient_drug_0_openfda_rxcui_24,patient_drug_0_openfda_product_ndc_1,patient_drug_0_openfda_product_ndc_2,patient_drug_0_openfda_product_ndc_3,patient_drug_0_openfda_product_ndc_4,patient_drug_0_openfda_product_ndc_5,patient_drug_0_openfda_product_ndc_6,patient_drug_0_openfda_product_ndc_7,patient_drug_0_openfda_product_ndc_8,patient_drug_0_openfda_product_ndc_9,patient_drug_0_openfda_product_ndc_10,patient_drug_0_openfda_product_ndc_11,patient_drug_0_openfda_package_ndc_3,patient_drug_0_openfda_package_ndc_4,patient_drug_0_openfda_package_ndc_5,patient_drug_0_openfda_package_ndc_6,patient_drug_0_openfda_package_ndc_7,patient_drug_0_openfda_package_ndc_8,patient_drug_0_openfda_package_ndc_9,patient_drug_0_openfda_package_ndc_10,patient_drug_0_openfda_package_ndc_11,patient_drug_0_drugadditional,patient_summary_narrativeincludeclinical,patient_patientonsetageunit,patient_drug_0_openfda_manufacturer_name_1,patient_drug_0_openfda_spl_set_id_1,patient_drug_0_openfda_spl_id_1,patient_drug_0_openfda_package_ndc_12,patient_drug_0_openfda_package_ndc_13,seriousnessdeath,patient_drug_0_drugrecur

In [7]:
#json_normalize(record)
json_normalize(records)

,receivedate,receivedateformat,transmissiondate,fulfillexpeditecriteria,occurcountry,receiptdate,transmissiondateformat,receiptdateformat,duplicate,reporttype,serious,safetyreportversion,companynumb,safetyreportid,primarysourcecountry,patient.reaction,patient.patientsex,patient.drug,sender.senderorganization,sender.sendertype,primarysource.reportercountry,primarysource.qualification,reportduplicate.duplicatenumb,reportduplicate.duplicatesource,receiver.receiverorganization,receiver.receivertype,seriousnessother,patient.patientonsetage,patient.summary.narrativeincludeclinical,patient.patientonsetageunit,seriousnessdeath,seriousnesshospitalization
0,20190227,102,20190418,2,US,20190227,102,102,1,1,2,1,US-NOVOPROD-607864,16013244,US,[{'reactionmeddrapt': 'Gastrointestinal disord...,2,"[{'medicinalproduct': 'SAXENDA', 'drugindicati...",FDA-Public Use,2,US,3,US-NOVOPROD-607864,NOVO NORDISK,FDA,6,NaN,NaN,NaN,NaN,NaN,NaN
1,20190227,102,20190418,1,US,20190301,102,102,1,1,1,2,US-PFIZER INC-2019082983,16013864,US,"[{'reactionmeddrapt': 'Growth retardation', 'r...",1,"[{'medicinalproduct': 'GENOTROPIN', 'drugindic...",FDA-Public Use,2,US,1,US-PFIZER INC-2019082983,PFIZER,FDA,6,1,3,CASE EVENT DATE: 20180624,801,NaN,NaN
2,20190227,102,20190418,1,DO,20190227,102,102,1,4,1,1,DO-ANIPHARMA-2019-DO-000001,16014487,DO,"[{'reactionmeddrapt': 'Death', 'reactionmeddra...",1,"[{'medicinalproduct': 'ATACAND', 'drugindicati...",FDA-Public Use,2,DO,NaN,DO-ANIPHARMA-2019-DO-000001,ANI,FDA,6,NaN,NaN,NaN,NaN,1,NaN
3,20190213,102,20190417,1,GB,20190213,102,102,1,1,1,1,GB-NOVOPROD-646035,15957136,GB,"[{'reactionmeddrapt': 'Diabetic ketoacidosis',...",2,"[{'medicinalproduct': 'NOVORAPID', 'drugindica...",FDA-Public Use,2,GB,5,GB-NOVOPROD-646035,NOVO NORDISK,FDA,6,NaN,42,CASE EVENT DATE: 20190202,801,NaN,1
4,20190213,NaN,NaN,NaN,ES,NaN,NaN,NaN,1,1,1,1,NaN,NaN,ES,[{'reactionmeddrapt': 'Premature separation of...,NaN,"[{'medicinalproduct': 'PROPESS', 'drugindicati...",NaN,NaN,ES,1,ES-FERRINGPH-2019FE00550,FERRING,FDA,6,1,NaN,NaN,NaN,NaN,NaN


In [8]:
# This is better 
# json_normalize(flat_dic)
json_normalize(flat_dics)

,receivedate,receivedateformat,patient.reaction.0.reactionmeddrapt,patient.reaction.0.reactionmeddraversionpt,patient.reaction.0.reactionoutcome,patient.patientsex,patient.drug.0.medicinalproduct,patient.drug.0.drugindication,patient.drug.0.drugcharacterization,patient.drug.0.drugadministrationroute,patient.drug.0.drugseparatedosagenumb,patient.drug.0.drugstructuredosageunit,patient.drug.0.openfda.manufacturer_name.0,patient.drug.0.openfda.unii.0,patient.drug.0.openfda.product_type.0,patient.drug.0.openfda.rxcui.0,patient.drug.0.openfda.rxcui.1,patient.drug.0.openfda.spl_set_id.0,patient.drug.0.openfda.route.0,patient.drug.0.openfda.generic_name.0,patient.drug.0.openfda.brand_name.0,patient.drug.0.openfda.product_ndc.0,patient.drug.0.openfda.substance_name.0,patient.drug.0.openfda.spl_id.0,patient.drug.0.openfda.application_number.0,patient.drug.0.openfda.package_ndc.0,patient.drug.0.openfda.package_ndc.1,patient.drug.0.openfda.package_ndc.2,patient.drug.0.drugbatchnumb,patient.drug.0.drugintervaldosageunitnumb,patient.drug.0.drugdosagetext,patient.drug.0.actiondrug,patient.drug.0.activesubstance.activesubstancename,patient.drug.0.drugintervaldosagedefinition,patient.drug.0.drugauthorizationnumb,patient.drug.0.drugdosageform,patient.drug.0.drugstructuredosagenumb,sender.senderorganization,sender.sendertype,transmissiondate,fulfillexpeditecriteria,occurcountry,receiptdate,transmissiondateformat,receiptdateformat,primarysource.reportercountry,primarysource.qualification,duplicate,reporttype,reportduplicate.duplicatenumb,reportduplicate.duplicatesource,receiver.receiverorganization,receiver.receivertype,serious,safetyreportversion,companynumb,safetyreportid,primarysourcecountry,seriousnessother,patient.reaction.1.reactionmeddrapt,patient.reaction.1.reactionmeddraversionpt,patient.reaction.1.reactionoutcome,patient.reaction.2.reactionmeddrapt,patient.reaction.2.reactionmeddraversionpt,patient.reaction.2.reactionoutcome,patient.reaction.3.reactionmeddrapt,patient.reaction.3.reactionmeddraversionpt,patient.reaction.3.reactionoutcome,patient.reaction.4.reactionmeddrapt,patient.reaction.4.reactionmeddraversionpt,patient.reaction.4.reactionoutcome,patient.patientonsetage,patient.drug.0.openfda.rxcui.2,patient.drug.0.openfda.rxcui.3,patient.drug.0.openfda.rxcui.4,patient.drug.0.openfda.rxcui.5,patient.drug.0.openfda.rxcui.6,patient.drug.0.openfda.rxcui.7,patient.drug.0.openfda.rxcui.8,patient.drug.0.openfda.rxcui.9,patient.drug.0.openfda.rxcui.10,patient.drug.0.openfda.rxcui.11,patient.drug.0.openfda.rxcui.12,patient.drug.0.openfda.rxcui.13,patient.drug.0.openfda.rxcui.14,patient.drug.0.openfda.rxcui.15,patient.drug.0.openfda.rxcui.16,patient.drug.0.openfda.rxcui.17,patient.drug.0.openfda.rxcui.18,patient.drug.0.openfda.rxcui.19,patient.drug.0.openfda.rxcui.20,patient.drug.0.openfda.rxcui.21,patient.drug.0.openfda.rxcui.22,patient.drug.0.openfda.rxcui.23,patient.drug.0.openfda.rxcui.24,patient.drug.0.openfda.product_ndc.1,patient.drug.0.openfda.product_ndc.2,patient.drug.0.openfda.product_ndc.3,patient.drug.0.openfda.product_ndc.4,patient.drug.0.openfda.product_ndc.5,patient.drug.0.openfda.product_ndc.6,patient.drug.0.openfda.product_ndc.7,patient.drug.0.openfda.product_ndc.8,patient.drug.0.openfda.product_ndc.9,patient.drug.0.openfda.product_ndc.10,patient.drug.0.openfda.product_ndc.11,patient.drug.0.openfda.package_ndc.3,patient.drug.0.openfda.package_ndc.4,patient.drug.0.openfda.package_ndc.5,patient.drug.0.openfda.package_ndc.6,patient.drug.0.openfda.package_ndc.7,patient.drug.0.openfda.package_ndc.8,patient.drug.0.openfda.package_ndc.9,patient.drug.0.openfda.package_ndc.10,patient.drug.0.openfda.package_ndc.11,patient.drug.0.drugadditional,patient.summary.narrativeincludeclinical,patient.patientonsetageunit,patient.drug.0.openfda.manufacturer_name.1,patient.drug.0.openfda.spl_set_id.1,patient.drug.0.openfda.spl_id.1,patient.drug.0.openfda.package_ndc.12,patient.drug.0.openfda.package_ndc.13,seriousnessdeath,patient.drug.0.drugrecur

***Using the tool below, I executed RestAPI queries to have a sense of what data change over time***

REMARKS:
Special treament is needed for certain scenarios

    1. There can be some drug indications that use different descriptions for the same purpose, e.g. patient.drug.drugindication.exact
    
    2. primarysource : ~ who reports
    
    primarysourcecountry only use country code, e.g. UK; 
    while primarysource.reportercountry can use either country code or full name (e.g. ) 
    
    NOTE: primarysourcecountry is not available in FDA website interactive tool
    https://open.fda.gov/apis/drug/event/explore-the-api-with-an-interactive-chart/ [who reports]
    
    primarysource.reportercountry:
        Country from which the report was submitted
    primarysourcecountry
        Country of the reporter of the event
    occurcountry
        The name of the country where the event occurred.
    
    A report through a company when "_exists_:companynumb" 
    A report by public when "_missing_:companynumb"
    
    Data are reported and consider autonomous regions explicitly. So there are reports coming from Åland Islands (AX), and also from Finland (FI).
    
    3. certain fields use index, and needs to map to a meaningful name, e.g. 
    
    primarysource.qualification ; 
    
    patient.drug.actiondrug (this is one possible choice of 'patient.drug' which is 'type=array')
    
    patient.drug.drugadministrationroute (the value now can be '001', '002' (str), rather than (int) 0, 1)
    
    4. some index-mapping does not have associated value
    
    patient.drug.drugstructuredosageunit 
    patient.drug.drugcumulativedosageunit [no key for '032']
    --> in this case, keep the 'key'
    
    5. certain fields have not-well-defined name, e.g. 
    
    patient.drug.drugbatchnumb [some can be considered the same, e.g. UNKNOWN, UNK, UNCONFIRMED
    ; or not sure what it means, e.g. SBFW]
    
    6. datetime format inconsistent
    
    patient.drug.drugstartdate 
    patient.drug.drugenddate [ some provides only 'year', e.g. 2013, while some provide very long number, e.g. 1301616000000] 
    
    patient.drug.drugenddateformat [ should be 102 (YYYYMMDD), but other values also used 610, 602 - not sure what format is this?]
    https://www.sqlservertutorial.net/sql-server-system-functions/convert-datetime-to-string/
    
    7. some fields may have unclear value 
    
    patient.drug.drugintervaldosage [e.g. "Total", "Cyclical" ...]
    
    8. some fields whose values seem missing, i.e. no available input from any records
    
    patient.drug.patientagegroup
    receiver
    
    9. possible of duplicated report?
    
    reportduplicate.duplicatenumb
    
    10. "receivedate" vs. "transmissiondate" vs. "receiptdate"
    
    There are three information about the date provided. 
    When testing with "receivedate", the records in 2019 Q1 can have "receivedate" in 2018, so it's not sure what is the criteria for 
    putting one record in each file. It turns out "receiptdate" is used [show later].


In [9]:
def f_extra_terms(x, exact, map_name, receivedate, extra_terms=""):
    data = query_count_drug_indication(count_field=x, exact=exact, receivedate=receivedate, extra_terms=extra_terms)
    
    out = []
    exist = {}
    index = 0
    terms = []
    # NAIVE result
    for result in data.get('results', []):
      # print(result)
      # RESULT RESULT: consider the value is an integer
      # .. mapping to a string name, e.g. primarysource.qualification
      if (map_name):
          result['term'] = map_index_to_value(attr_list, x, result['term'])
          #print(type(result['term'])) 
      terms.append(result['term'])
    if len(terms) == 0:
        return "No record found"

    #x = np.array(terms)
    #print(len(terms))
    #print(len(np.unique(x)))
    #pp.pprint(terms)

    # REVISE RESULT
    # ... CHECK for overlapping description
    for result in data.get('results', []):
        if str(result['term']).find("FOR UNKNOWN INDICATION") >= 0:
            result['term'] = "UNKNOWN INDICATION"
            if "UNKNOWN INDICATION" not in exist:
                exist["UNKNOWN INDICATION"] = index
                index = index + 1
                out.append(result)
            else:
             out[exist["UNKNOWN INDICATION"]]["count"] += result["count"]
        else:
            out.append(result)
    # print(json.dumps(out, indent=1))
    df = pd.DataFrame(out)
    print(df)
    # plt.figure(figsize=(16, 6))
    sns.set(rc={'figure.figsize':(16,6)})
    g = sns.barplot(x='term', y="count", data=df[0:12], palette=pkmn_type_colors)

    # Rotate x-labels
    g.set_xticklabels(g.get_xticklabels(), rotation=-45, fontsize=10)
    # return out # data["results"]
    import time
    file_name = time.strftime("%Y%m%d-%H%M%S")
    g.figure.savefig("images/%s.png" % file_name, bbox_inches="tight")
    df.to_pickle("images/%s.pkl" % file_name)
    
def f(x, exact, map_name, receivedate):
    extra_terms=""
    f_extra_terms(x, exact, map_name, receivedate, extra_terms)
    

In [10]:
# Remember that this only returns records in that the field is present. There are several records in that certain fields are missing.
interact(f, x=list(searchable_fields), 
         exact=False, map_name=False, receivedate="20040101+TO+20160601")

interactive(children=(Dropdown(description='x', options=('authoritynumb', 'companynumb', 'duplicate', 'fulfill…

<function __main__.f(x, exact, map_name, receivedate)>

In [11]:
# Are adverse events coming from different countries?
# --> test with 'primarysource.reportercountry'
# .    "primarysourcecountry"
# .    "occurcountry"
interact(f, x=widgets.Dropdown(options=list(searchable_fields), value="primarysourcecountry"), 
         exact=True, map_name=False, receivedate="20040101+TO+20191201")

interactive(children=(Dropdown(description='x', index=63, options=('authoritynumb', 'companynumb', 'duplicate'…

<function __main__.f(x, exact, map_name, receivedate)>

In [12]:
# Are adverse events associated with different disease areas?
list_terms = [ ("", ""), ("female", "patient.patientsex:2"), ("male", "patient.patientsex:1")]
interact(f_extra_terms, x=widgets.Dropdown(options=list(searchable_fields), value="patient.drug.drugindication"), 
         exact=True, map_name=False, 
         extra_terms=list_terms, receivedate="20040101+TO+20160601")

interactive(children=(Dropdown(description='x', index=18, options=('authoritynumb', 'companynumb', 'duplicate'…

<function __main__.f_extra_terms(x, exact, map_name, receivedate, extra_terms='')>

Disease areas can be found in 

* patient.drug.drugindication

Drug names can be found in 

* patient.drug.medicinalproduct

    This may be the valid trade name of the product (such as `ADVIL` or `ALEVE`) or the generic name (such as `IBUPROFEN`). This field is not systematically normalized. It may contain misspellings or idiosyncratic descriptions of drugs, such as combination products such as those used for birth control.
    
* patient.drug.openfda.brand_name
    
    Brand or trade name of the drug product.
    
* patient.drug.openfda.generic_name
    
    Generic name of the drug product.
    
* patient.drug.openfda.nui
    Unique identifier applied to a drug concept within the National Drug File Reference Terminology (NDF-RT)."
    
* patient.drug.activesubstance.activesubstancename

    Product active ingredient, which may be different than other drug identifiers (when provided)
    
* patient.drug.drugcharacterization
    the result tells whether the drug is the main cause of the event [not validated by FDA]
     value:
                  '1': "Suspect (the drug was considered by the reporter to be the cause)"
                  '2': "Concomitant (the drug was reported as being taken along with the suspect drug)"
                  '3': "Interacting (the drug was considered by the reporter to have interacted with the suspect drug)"
    

## The below section is used for retrieving OpenFDA using Restful APIs

The data is saved to file "./request_openfda.json" file

NOTE: OpenFDA does not allow downloading more than 25000+99 records

```console
{
  "error": {
    "code": "BAD_REQUEST",
    "message": "Skip value must 25000 or less."
  }
}
```

In [13]:
### Rather than using data from downloaded files, we can do query 
### to get recorded we need
JSON_FILENAME = './requested_openfda.json'

#from request_openfda import query_count_drug_events 
pd.set_option('display.max_columns', None)
def recursive_items(dictionary):
    print(type(dictionary))
    if type(dictionary) is dict:
        for key, value in dictionary.items():
            if type(value) is dict:
                yield from recursive_items(value)
            else:
                yield (key, value)
            
def f(x, find_all_country, limit, skip, find_all, receivedate):
    if find_all_country:
        code = ""
    else:
        code = df_country_code.loc[df_country_code["Name"] == x]["Code"].values[0]
    if find_all:
        data = query_drug_events(receivedate=receivedate, country=code, skip=skip, find_all=True)
    else:
        data = query_drug_events(receivedate=receivedate, country=code, limit=limit, skip=skip)
        
    if data is None:
        # do nothing
        return 
    
    with open(JSON_FILENAME, 'w') as outfile:
        json.dump(data, outfile)

    out = []
    exist = {}
    index = 0
    terms = []
    # NAIVE result
    #pp.pprint(data["results"])
    # for key, value in recursive_items(data["results"]):
    #     print("KEY ", key)
    from pandas.io.json import json_normalize
    terms = json_normalize(data["results"], max_level=1)
    # from IPython.display import display
    # from IPython.core.display import HTML
# 
    # display(HTML(terms.to_html()))
    #pd.options.display.max_columns = None
    #display(terms)

    # for result in data.get('results', []):
    #   # print(result)
    #   terms.append(result)
    # print("Total records returned: ", len(terms))
    print(terms.columns.values)
    return terms
   
interact_manual(f, x=list(df_country_code["Name"].to_list()), 
         find_all_country=False,
         limit=widgets.IntSlider(min=1, max=99, step=1, value=1),
         skip=widgets.IntSlider(min=0, max=50, step=10, value=0),
         find_all=False, receivedate="20040101+TO+20190601")
    

interactive(children=(Dropdown(description='x', options=('Afghanistan', 'Åland Islands', 'Albania', 'Algeria',…

<function __main__.f(x, find_all_country, limit, skip, find_all, receivedate)>


**Next stage?**

As a guide, you might consider a practical solution to one of the following
questions: 
 
 Are different adverse events reported in different countries? 

 What are the different adverse events associated with different disease areas? 

 What drugs tend to be taken together?

---